In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
!pip install -U transformers
!pip install faker --quiet

In [ ]:
!pip install accelerate
!pip install bitsandbytes

In [1]:
# login required for gemma usage
from huggingface_hub import notebook_login

notebook_login()

In [52]:
import torch
import os

CUDA_VISIBLE_DEVICES = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(CUDA_VISIBLE_DEVICES)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [53]:
import sys, random, string, re, time
import pandas as pd
import numpy as np
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm.auto import tqdm
from faker import Faker  #generates fake data 
from spacy.lang.en import English

In [54]:
import torch, random
# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

SEED = 42
# Seed the same seed to all 
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(SEED)

In [55]:
import ctypes, gc
libc = ctypes.CDLL("libc.so.6")
def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()

**GENERATE FAKE STUDENT INFORMATION**

In [8]:
# Random generate 12 random number
def get_userid(length=16):
    """Generate userid - """
    userid = str(int(np.random.rand()*1_000_000_000))
    # Add the extra rand chars
    for _ in range(length):
        # Select random char or digital number (0-9)
        userid = userid + random.choice(string.ascii_letters + str(random.randint(0, 9))) 
    return userid

# Generate the personal url from social media 
def generate_fake_social_media_url(user_name):
    social_media_platforms = {
        'LinkedIn': 'linkedin.com/in/',
        'YouTube': 'youtube.com/c/',
        'Instagram': 'instagram.com/',
        'GitHub': 'github.com/',
        'Facebook': 'facebook.com/',
        'Twitter': 'twitter.com/'
    }
    _ , domain = random.choice(list(social_media_platforms.items()))
    fake_url = f'https://{domain}{user_name}'
    return fake_url

def generate_username(first_name, last_name, fake_user_name):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]    
    if random.choice([False, True]):
        username = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}{random.randint(1,999)}"
    else:
        username = fake_user_name
    return username

def generate_email(first_name, last_name, faker):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]
    if random.choice([False, True]):
        email = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}@{faker.domain_name()}"
    else:
        email = faker.email()
    return email

def generate_student_info():
    """Generates all the user info (name, eamil addresses, phone number, etc) together """
    # Select the student country to generate the user info based on the country
    COUNTRIES = ["en_US", "en_US", "en_US", "en_US", "en_US", "en_US",
                 "de_DE", "it_IT", "es_ES", "da_DK", "fr_FR", "vi_VN"]
    country = random.choice(COUNTRIES)
    faker = Faker(country)
    first_name = faker.first_name()
    last_name = faker.last_name()
    user_name = generate_username(first_name, last_name, faker.user_name())
    fake_url = generate_fake_social_media_url(user_name)
    student = {}
    student['COUNTRY'] = country
    student['ID_NUM'] = get_userid() # User ID
    student['NAME_STUDENT'] = first_name + " "+  last_name 
    student['EMAIL'] = generate_email(first_name, last_name, faker)
    student['USERNAME'] = user_name
    student['PHONE_NUM'] = faker.phone_number().replace(" ", "")
    student['URL_PERSONAL'] = fake_url
    student['STREET_ADDRESS'] = str(faker.address()).replace("\n"," ") # Replace \n with space in the address
    del faker
    clear_memory()
#     print(student)
    return student   

In [9]:
from pathlib import Path
# Create the folder
Path("/home/korzanova/pii").mkdir(parents=True, exist_ok=True)
LOADED = False # True:  Load the pre-generated student info
              # False: Generate a new collection of student info    
if LOADED:
    df = pd.read_csv("/home/korzanova/pii/ai-generated-text-dataset/df_generated.csv")
    df.to_csv("/home/korzanova/pii/df_generated.csv", index=False, encoding='UTF-8')
    display(df.tail(5))
else:
    TOTAL = 100 # Generate 10,000
    students = []
    for _ in tqdm(range(TOTAL)):
        students.append(generate_student_info())
    df = pd.DataFrame(students)
    df = df.reset_index(drop=True)
    # Save to the csv file
    df.to_csv("/home/korzanova/pii/df_generated.csv", index=False, encoding='UTF-8') # Do not save default ID column
    display(df.tail(10))
    # Check if ID_NUM has any duplicates
    assert df['ID_NUM'].duplicated().value_counts()[False] == TOTAL, "Duplicated ID_NUM"
    sys.exit(0)

  0%|          | 0/100 [00:00<?, ?it/s]

,COUNTRY,ID_NUM,NAME_STUDENT,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
90,en_US,119594245AiylOSJFiuNwFmJP,Courtney Schneider,courtney.schneider@mercado.com,williamnunez,+1-807-357-8349x815,https://facebook.com/williamnunez,"61490 Riley Gardens Apt. 842 Jenniferton, GU 3..."
91,fr_FR,713244787kKYsrIkKJhhX8IWI,Jules Chevallier,isaac06@example.net,jules.chevallier728,0164672607,https://github.com/jules.chevallier728,"42, avenue de Charpentier 94974 Charrier-sur-B..."
92,de_DE,760785048tCwCXnSTOki3ZKIM,Ludmilla Lange,freudenbergerluzie@example.org,ilkahecker,02603934971,https://twitter.com/ilkahecker,Hamannallee 7 55283 Stade
93,en_US,561277197RidPSpTpFKOyezVS,Tammy Vang,tammy.vang@garcia.com,tammy.vang587,841.647.8231x90785,https://instagram.com/tammy.vang587,"5009 Dickson Lakes North Andrea, NJ 70318"
94,it_IT,770967179UOezBFuvKKYRcObk,Caterina Migliaccio,caterina.migliaccio@granatelli-chiesa.it,caterinamigliaccio3,+39093417874,https://twitter.com/caterinamigliaccio3,"Rotonda Geraci, 3 Piano 9 30133, Sacca Fisola ..."
95,en_US,493795596kfqxdOUBTaEioRaS,Sean Rodriguez,sean_rodriguez@fletcher.biz,sean_rodriguez706,(312)241-6450x196,https://youtube.com/c/sean_rodriguez706,"PSC 8908, Box 2505 APO AP 62799"
96,de_DE,522732829ofgxiXPeKNCUOARn,Kreszentia Kitzmann,carlos35@example.com,cscheel,0859132745,https://github.com/cscheel,Heinrichstr. 88 85471 Biedenkopf
97,en_US,427541018UwupPpMUsQnMmsYF,Meredith Garcia,meredith_garcia@villarreal.biz,amanda39,392.641.2946x2088,https://linkedin.com/in/amanda39,"16383 Shelly Terrace Suite 755 Wesleystad, MN ..."
98,en_US,25419126auyAIWXbeqcYGOXB,Christopher King,christopher_king@murphy.info,christopherking609,965-367-0380,https://linkedin.com/in/christopherking609,"398 Sarah Estate Moonfort, MH 77571"
99,fr_FR,107891426tGZIzOtUdfhNdQhT,Benjamin Fischer,morinbernard@example.net,laurentluce,0493542192,https://instagram.com/laurentluce,"280, boulevard de Lopes 78810 RocheBourg"


SystemExit: 0

/home/korzanova/miniconda3/envs/py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


****ESSAY GENERATION WITH OPENCHAT****

In [ ]:
df_generated = pd.read_csv("df_generated.csv")

In [43]:
import requests, json

In [32]:
def generate_on_user_essay(row):
    # Prompt template generates the texts
    prompt_template = """
    You are {first_name} {last_name}. 
    Write a short essay as your final assignment for course on critical thinking. 
    Mention your personal info:
    email {email}, address {address}, phone number {phone_num}, personal url {url}, username {username} and user id {userid}
    Mention all of the above. Write an essay about 420 words long. 
    Begin with the title **Essay**"""
    
    first_name = row['NAME_STUDENT'].split()[0]
    last_name = row['NAME_STUDENT'].split()[1]
    email = row['EMAIL']
    phone_num = row['PHONE_NUM']
    address = row['STREET_ADDRESS']
    url = row['URL_PERSONAL']
    username = row['USERNAME']
    userid = row['ID_NUM']
    # Fill in prompt with PII
    prompt = prompt_template.format(first_name=first_name,
                                    last_name=last_name,
                                    email=email,
                                    phone_num=phone_num,
                                    address=address,
                                    url=url,
                                    username=username,
                                    userid=userid
                                   )
    return prompt

In [33]:
def generate_essays(model, df, start_pos, end_pos, df_to_save):
    generated_df = df[start_pos:end_pos]
    generated_df = generated_df.reset_index(drop=True)
    generated_texts = []
    headers = {"Content-Type": "application/json"}
    url = "http://localhost:18888/v1/chat/completions"
    # Generate the texts
    for i in tqdm(range(len(generated_df))):
        start = time.time()
        row = generated_df.loc[i]
        try:
            # Generate the texts using the prompts
            prompt = generate_on_user_essay(row)
            # print(f"prompt = {prompt}")
            data = {"model": model, "messages": [{"role": "user", "content": prompt}]}
            response = requests.post(url, data=json.dumps(data), headers=headers)
            content = json.loads(response.content)
            generated_text = content["choices"][0]["message"]["content"]
            generated_texts.append(generated_text)
            # print(f"generated_text = {generated_text}" )
        except Exception as e:
            print(f"Error occurs during text generate {e}")
            generated_texts.append("Errors")
        clear_memory()
        print(f"Complete the text for {i}-th student {time.time() - start: .1f} seconds")
    assert len(generated_df) == len(generated_texts), "Generate texts length is mismatched"
    # Store all the generated texts
    generated_df['generated_text'] = generated_texts
    final_df = pd.concat([df_to_save, generated_df], ignore_index=True) # to concat two DFs with essays on different topics
    # Save generated_df to csv
    final_df.to_csv(f"generated_df_with_essays_final.csv", index=False, encoding="UTF-8")

In [ ]:
MODEL = "openchat_3.5"
num_essays = 50
start_pos = 50
end_pos = start_pos + num_essays
df_to_save = pd.read_csv("generated_df_with_essays.csv") # df in-between to concat with eaays on other topic

# Generate the texts
generate_essays(MODEL, df_generated, start_pos, end_pos, df_to_save=df_to_save)

In [70]:
df = pd.read_csv('generated_df_with_essays_final.csv')

In [71]:
df.tail(5)

,COUNTRY,ID_NUM,NAME_STUDENT,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS,generated_text
95,en_US,493795596kfqxdOUBTaEioRaS,Sean Rodriguez,sean_rodriguez@fletcher.biz,sean_rodriguez706,(312)241-6450x196,https://youtube.com/c/sean_rodriguez706,"PSC 8908, Box 2505 APO AP 62799",Title: Essay\n\nAs a student of critical think...
96,de_DE,522732829ofgxiXPeKNCUOARn,Kreszentia Kitzmann,carlos35@example.com,cscheel,0859132745,https://github.com/cscheel,Heinrichstr. 88 85471 Biedenkopf,Essay\n\nTitle: The Importance of Critical Thi...
97,en_US,427541018UwupPpMUsQnMmsYF,Meredith Garcia,meredith_garcia@villarreal.biz,amanda39,392.641.2946x2088,https://linkedin.com/in/amanda39,"16383 Shelly Terrace Suite 755 Wesleystad, MN ...",Essay\n\nTitle: The Importance of Critical Thi...
98,en_US,25419126auyAIWXbeqcYGOXB,Christopher King,christopher_king@murphy.info,christopherking609,965-367-0380,https://linkedin.com/in/christopherking609,"398 Sarah Estate Moonfort, MH 77571",Essay\n\nTitle: The Importance of Critical Thi...
99,fr_FR,107891426tGZIzOtUdfhNdQhT,Benjamin Fischer,morinbernard@example.net,laurentluce,0493542192,https://instagram.com/laurentluce,"280, boulevard de Lopes 78810 RocheBourg","Essay\n\nAs Benjamin Fischer, I am pleased to ..."


In [5]:
# making a small dataset to test labelling on
test_df = df[:2]
test_df

,COUNTRY,ID_NUM,NAME_STUDENT,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS,generated_text
0,fr_FR,374540118piRfBbnGbmACrWSv,Antoinette Hamon,antoinette_hamon@dupre.org,jpons,0564596965,https://linkedin.com/in/jpons,"47, rue de Grondin 32275 Saint Adrienne",Essay\n\nMy Journey Through the World of Story...
1,en_US,950714306wMZUIyJOxmcXfgrO,Nicholas Rosales,nicholas_rosales@morales-rodriguez.com,nicholas_rosales95,968-319-5485,https://github.com/nicholas_rosales95,"1694 Dunn Road Suite 180 Fryeside, SD 39082",Essay\n\nTitle: The Journey of Self-Discovery:...


In [8]:
test_df["STREET_ADDRESS"]

0        47, rue de Grondin 32275 Saint Adrienne
1    1694 Dunn Road Suite 180 Fryeside, SD 39082
Name: STREET_ADDRESS, dtype: object

**LABELLING**

In [72]:
from spacy.lang.en import English
en_tokenizer = English().tokenizer

def tokenize_with_spacy(text, tokenizer=en_tokenizer):
    tokenized_text = tokenizer(text)
    tokens = [token.text for token in tokenized_text]
    trailing_whitespace = [bool(token.whitespace_) for token in tokenized_text]
    return {'tokens': tokens, 'trailing_whitespace': trailing_whitespace}

In [ ]:
!pip install these-united-states

In [73]:
import united_states

In [74]:
us = united_states.UnitedStates()
state_dict = {s.abbr : s.name for s in us.states}

In [75]:
def entities_to_label_dict(row):
    label_dict={"STREET_ADDRESS": [row["STREET_ADDRESS"]],
                "NAME_STUDENT": [row["NAME_STUDENT"]],
                "EMAIL": [row["EMAIL"]],
                "PHONE_NUM": [row["PHONE_NUM"]],
                "ID_NUM": [row["ID_NUM"]],
                "USERNAME": [row["USERNAME"]],
                "URL_PERSONAL": [row["URL_PERSONAL"]]}
    return label_dict

In [76]:
labels = []
for _, row in df.iterrows():
    row_labels = entities_to_label_dict(row)
    labels.append(row_labels)

df["label_dict"] = labels

In [78]:
# check text for labelling
df["label_dict"][15]

{'STREET_ADDRESS': ['26615 Jason Knolls South Rodneyview, PR 18154'],
 'NAME_STUDENT': ['Thomas Bowen'],
 'EMAIL': ['thomas_bowen@sanchez-ochoa.info'],
 'PHONE_NUM': ['254-379-1477x0289'],
 'ID_NUM': ['183404509bpNDOmFpPWXoZDJh'],
 'USERNAME': ['thomas_bowen826'],
 'URL_PERSONAL': ['https://youtube.com/c/thomas_bowen826']}

In [79]:
def pj_to_pj(row):
    # wanted format: {"NAME_STUDENT":["Valentin", "Werner"]}
    labeldict={}
    labels_ = row["label_dict"]
    # Split non US type of streets (e.g., german streets)

    for k, v in labels_.items():
        v_processed=[]
        
        for x in v:
            if k == "STREET_ADDRESS":
                x = re.split("-", x)
                x = " ".join(x)

            x = x.replace("\n"," ")
            x = re.sub(" +", " ", x)
            # This splits entities such as "Aaron Smith" to "Aaron", "Smith"
            x = x.split(" ")
            for n in x:
                try:
                    while n[-1] in string.punctuation: 
                        n=n[:-1]
                except: print(n)
                v_processed+=[n]
                        
            # Map country shorts to fullname
            if k == "STREET_ADDRESS":
                for x in v_processed:
                    if x in state_dict.keys():
                        v_processed+=state_dict[x].split(" ")

        labeldict[k] = v_processed
    return labeldict

In [80]:
tokens = [tokenize_with_spacy(r["generated_text"]) for _, r in df.iterrows()]
labels_=[]
for _, row in df.iterrows():
    labeldict = pj_to_pj(row)
    labels_.append(labeldict)

labels_

[{'STREET_ADDRESS': ['47',
   'rue',
   'de',
   'Grondin',
   '32275',
   'Saint',
   'Adrienne'],
  'NAME_STUDENT': ['Antoinette', 'Hamon'],
  'EMAIL': ['antoinette_hamon@dupre.org'],
  'PHONE_NUM': ['0564596965'],
  'ID_NUM': ['374540118piRfBbnGbmACrWSv'],
  'USERNAME': ['jpons'],
  'URL_PERSONAL': ['https://linkedin.com/in/jpons']},
 {'STREET_ADDRESS': ['1694',
   'Dunn',
   'Road',
   'Suite',
   '180',
   'Fryeside',
   'SD',
   '39082',
   'South',
   'Dakota'],
  'NAME_STUDENT': ['Nicholas', 'Rosales'],
  'EMAIL': ['nicholas_rosales@morales-rodriguez.com'],
  'PHONE_NUM': ['968-319-5485'],
  'ID_NUM': ['950714306wMZUIyJOxmcXfgrO'],
  'USERNAME': ['nicholas_rosales95'],
  'URL_PERSONAL': ['https://github.com/nicholas_rosales95']},
 {'STREET_ADDRESS': ['76007',
   'Cervantes',
   'Greens',
   'East',
   'Janettown',
   'OR',
   '12395',
   'Oregon'],
  'NAME_STUDENT': ['Marc', 'Thomas'],
  'EMAIL': ['kevin42@example.com'],
  'PHONE_NUM': ['681.282.3799x5748'],
  'ID_NUM': ['73199

In [ ]:
for d in labels_:
    d["PHONE_NUM"] = [t for t in tokenize_with_spacy(d["PHONE_NUM"][0])["tokens"] if t not in string.punctuation]
    print(d["PHONE_NUM"])

In [82]:
def flag_main_ents(tokens, labels_):
    """Looks for entities from the labels dictionary inside the text and flags their position"""
    new_=[]
    for tok, l in zip(tokens, labels_):
        
        # these will just be forwarded to the final result, as we do not change these
        t = tok["tokens"]
        ws = tok["trailing_whitespace"]
        
        # Create "O" label as standard value to overwrite on specific indices
        new_labels=["O"]*len(t)
        
        # Find entities from labels_ in the text
        for ent_type, ent_list in l.items():
            for ent_ in ent_list:
                # find occurence of tagged entities in the list
                # - this assumes that entities are not containing commong words such as "the"
                indices = [i for i, x in enumerate(t) if x.lower() == ent_.lower()]
                for i in indices:
                    # overwrite "O" label with correct label
                    new_labels[i] = ent_type
        new_.append({"tokens":t, "trailing_whitespace":ws, "labels":new_labels})
    return new_

In [ ]:
new_ = flag_main_ents(tokens, labels_)
new_

In [84]:
sandwich_on_comma = ["STREET_ADDRESS"]
def fill_sandwiches(new_):
    """Looks for tokens, such as punctuation that were not flagged as label during flag_main_ents"""
    new_2=[]
    for r in new_:
        
        # again these are just forwarded
        t = r["tokens"]
        ws = r["trailing_whitespace"]
        # again, these may get overwritten
        label = r["labels"]
        new_labels=["O"]*len(label)
        for i, l in enumerate(label):
            # get prior label if possible
            if i != 0: prior_label=label[i-1]
            else: prior_label="O"

            # get next label
            if i+1 < len(label): next_label=label[i+1]
            elif i+1 == len(label): next_label="O"
            
            # skip filler / list words that split multiple entities
            if (t[i] == "and" and l == "O") or (t[i] == "or" and l == "O"):
                new_labels[i] = "O"
            elif prior_label == "EMAIL" and t[i] == "to":
                new_labels[i] = "O"

            # NOTE: this filter would technically be great, but disabling it seems like the lesser evil (more FP > more FN)
            # elif prior_label == "STREET_ADDRESS" and t[i] == "in":
            #     new_labels[i] = "O"
            
            # only street addresses should contain commas - this avoids labelling sandwiches
            # which chain multiple entities, such as "Valentin Werner, Thomas Müller, and Manuel Neuer"
            # As these should be three separate entities
            elif t[i] == "," and prior_label not in sandwich_on_comma:
                new_labels[i] = "O"
                
            # replace if we got a sandwich ("LABEL"-"O"-"LABEL", such as "Berlin" - "," - "Germany")
            elif prior_label == next_label and prior_label != "O":
                new_labels[i] = prior_label
            # Maintain old label if not "O"
            elif l != "O": new_labels[i] = l
            else: new_labels[i] = "O"
        
        new_2.append({"tokens":t, "trailing_whitespace":ws, "labels":new_labels})
    return new_2

In [ ]:
new_2 = fill_sandwiches(new_)
new_2

In [86]:
def add_bio_tags(new_2, labels_):
    """Adds B-(Beginning) and I-(Intermediate) tags to entities"""
    new_bio=[]
    for idx, r in enumerate(new_2):
        
        # again, these are just forwarded
        t = r["tokens"]
        ws = r["trailing_whitespace"]
        # again, these might get overwritten
        label = r["labels"]

        # keep track of last label to identify when to use B or I
        last_label="O"

        # Loop over all labels and assign starter label
        for i, l in enumerate(label):
            if l != last_label and l != "O":
                label[i] = "B-"+l
            elif l == last_label and last_label != "O":
                label[i] = "I-"+l

            # special rule for address only, as it is important the street is included in the entitity
            if label[i] == "B-STREET_ADDRESS":
                # NOTE: i first had thiss at :1, to also use the street name without the number
                # but it is still unclear, whether the number needs to be involved
                # as we rather have FP than FN, I included redid it so it at least includes the street name
                if t[i].lower() not in [x.lower() for x in labels_[idx]["STREET_ADDRESS"][:2]]:
                    label[i] = "O"
                    l = "O"

            # special rule for phone only
            if label[i] == "B-PHONE_NUM":
                if t[i].lower() not in [x.lower() for x in labels_[idx]["PHONE_NUM"][:1]]:
                    label[i] = "O"
                    l = "O"

            last_label = l

        new_bio.append({"tokens":t, "trailing_whitespace":ws, "labels":label})
    
    return new_bio

In [ ]:
new_3 = add_bio_tags(new_2,labels_)
new_3

In [ ]:
df

In [89]:
def save_data(new_3, original, save_as):
    """Saves Data as CSV and JSON"""
    results = []
    doc_id = 1 # document id 
    for i in range(len(original)):
        row = original.iloc[i]
        full_text = row["generated_text"]
        # Tokenize the text using spacy tokenizer
        doc_id += 1
        result = {
            'document': doc_id, 
            'full_text': full_text
        }
        results.append(result)
    new_test_df = pd.DataFrame(results)
    # print(new_test_df)
    new_test_df = new_test_df.reset_index(names="doc_")[["doc_","document","full_text"]]
    df = pd.DataFrame(new_3).reset_index(names="doc_")

    x = pd.merge(df, new_test_df, on="doc_", how="left")
    x = x.drop(columns="doc_")[["document","full_text","tokens","trailing_whitespace","labels"]]
    x = x.explode(["tokens", "trailing_whitespace", "labels"])

    fixed = (
        x.groupby("document")
        .agg(
            {
                "full_text": lambda x: x,
                "tokens": lambda x: x.tolist(),
                "trailing_whitespace": lambda x: x.tolist(),
                "labels": lambda x: x.tolist(),
            }
        )
        .reset_index()
    )
    # fix text that was turned into list
    fixed["full_text"] = fixed.full_text.apply(lambda x: x[0])

    fixed.to_csv(f"{save_as}.csv", index = False)
    
    json_format=[]
    for idx, row in fixed.iterrows():
        doc=row["document"]
        text=row["full_text"]
        tokens=row["tokens"]
        ws=row["trailing_whitespace"]
        labels=row["labels"]
        
        json_format.append(
            {
                "document":doc,
                "full_text":text,
                "tokens":tokens,
                "trailing_whitespace":ws,
                "labels":labels
            }
        )
    
    out_file = open(f"{save_as}.json", "w") 
    json.dump(json_format, out_file) 
    out_file.close()

In [90]:
save_data(new_3, df, "labelled_generated_dataset")

In [91]:
df = pd.read_csv("labelled_generated_dataset.csv")
df

,document,full_text,tokens,trailing_whitespace,labels
0,2,Essay\n\nMy Journey Through the World of Story...,"['Essay', '\n\n', 'My', 'Journey', 'Through', ...","[False, False, True, True, True, True, True, T...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,3,Essay\n\nTitle: The Journey of Self-Discovery:...,"['Essay', '\n\n', 'Title', ':', 'The', 'Journe...","[False, False, False, True, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,4,Essay\n\nTitle: The Art of Storytelling: A Per...,"['Essay', '\n\n', 'Title', ':', 'The', 'Art', ...","[False, False, False, True, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,5,Title: Essay\n\nAs a final assignment for my s...,"['Title', ':', 'Essay', '\n\n', 'As', 'a', 'fi...","[False, True, False, False, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,6,"Title: Essay\n\nAs a storytelling enthusiast, ...","['Title', ':', 'Essay', '\n\n', 'As', 'a', 'st...","[False, True, False, False, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...,...
95,97,Title: Essay\n\nAs a student of critical think...,"['Title', ':', 'Essay', '\n\n', 'As', 'a', 'st...","[False, True, False, False, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
96,98,Essay\n\nTitle: The Importance of Critical Thi...,"['Essay', '\n\n', 'Title', ':', 'The', 'Import...","[False, False, False, True, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
97,99,Essay\n\nTitle: The Importance of Critical Thi...,"['Essay', '\n\n', 'Title', ':', 'The', 'Import...","[False, False, False, True, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
98,100,Essay\n\nTitle: The Importance of Critical Thi...,"['Essay', '\n\n', 'Title', ':', 'The', 'Import...","[False, False, False, True, True, True, True, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
